In [7]:
import pandas as pd
from itertools import combinations
from nltk.metrics.agreement import AnnotationTask
from IPython.display import HTML

In [4]:
def alpha_df(df, good_only=False):
    data = []
    questions = range(1, 128)
    if good_only: questions = list(set(questions)-set({i for i in questions if len(answers[i].dropna().unique()) > 3}))
    for annotator in df.index:
        for question in questions:
            cell = df[question][annotator]
            if cell is not None : data.append((annotator, question, cell))
    try: return AnnotationTask(data=data).alpha()
    except ZeroDivisionError: return 1

In [3]:
def get_trust_coefs(df, good_only=False):
    alphas = dict()
    for i in range(2, len(df.index)+1):
        for combination in combinations(df.index, i):
            alphas[combination] = alpha_df(df.loc[list(combination)], good_only)
    coefs = dict()
    coef = 1
    sorted_keys = [key for key, val in sorted(alphas.items(), key = lambda ele: ele[1])]
    for i, key in enumerate(sorted_keys):
        if alphas[key] > alphas[sorted_keys[i-1]] : coef += 1
        for annotator in key:
            try: coefs[annotator] += coef
            except KeyError: coefs[annotator] = coef
    m = max([coef for coef in coefs.values()])
    return {annotator : c/(coef+m) for annotator, c in coefs.items()}

In [4]:
def ident_bad_annot(coefs, good_only=False):
    for annotator, coef in coefs.items():
        if coef <= 0.5: yield annotator

In [2]:
answers = pd.read_csv("answers.csv", names = ["date_time", "email", "ling", "coref"]+list(range(1, 131))+["comments"])
answers.drop(0, axis=0, inplace=True)
answers.drop("date_time", axis=1, inplace=True)
answers = answers.groupby("email").last()
info, answers = answers[["ling", "coref"]], answers[range(1, 131)]

In [112]:
doubles = {23:128, 38:129, 75:130}
checks = {}
for annotator in answers.index:
    check = []
    for k, v in doubles.items():
        first, last = answers[k][annotator], answers[v][annotator]
        check.append("non vérifié") if None in (first, last) else check.append("approuvé") if first == last else check.append("désapprouvé: "+first+" vs "+last)
    checks[annotator] = check
pd.DataFrame(checks).T.style.applymap(lambda x: "background-color: honeydew" if x == "approuvé" else "background-color: white" if x == "non vérifié" else "background-color: lavenderblush")

,0,1,2
Frederique.bordignon@enpc.fr,non vérifié,non vérifié,non vérifié
anais.halftermeyer@univ-orleans.fr,non vérifié,non vérifié,non vérifié
bergler@cse.concordia.ca,non vérifié,non vérifié,non vérifié
emmanuel.schang@univ-orleans.fr,non vérifié,non vérifié,non vérifié
fanny.krimou@hotmail.fr,non vérifié,non vérifié,non vérifié
ilaine.wang@inalco.fr,désapprouvé: 5 vs 6,désapprouvé: 7 vs 6,approuvé
loic.grobol@gmail.com,non vérifié,non vérifié,non vérifié
mariya,approuvé,approuvé,approuvé
sylvie.billot@univ-orleans.fr,désapprouvé: 4 vs 2,approuvé,approuvé
ygor.gallina@univ-nantes.fr,non vérifié,non vérifié,non vérifié


In [5]:
smaller = {"2" : "1", "3" : "2", "4" : "2", "5" : "3", "6" : "3", "7" : "4"}
gravity = {"0":"grave", "1":"grave", "2":"grave", "3":"pas grave", "4":"pas grave", "5":"pas grave", "6":"pas grave", "7":"pas de fautes"}
centrality = {"0":"fondamental", "1":"périphérique", "2":"périphérique", "3":"fondamental", "4":"fondamental", "5":"périphérique", "6":"périphérique", "7":"pas de fautes"}
small, gravity_eval, centrality_eval = [answers.applymap(lambda x : scale[x] if x in scale.keys() else x) for scale in [smaller, gravity, centrality]]
scale_names = ["standard", "réduit","gravité", "centralité"]
alphas_krip = {"échelle standard": [alpha_df(answers), alpha_df(answers, good_only=True)],
              "échelle réduite": [alpha_df(small), alpha_df(small, good_only=True)],
              "échelle de gravité": [alpha_df(gravity_eval), alpha_df(gravity_eval, good_only=True)],
              "échelle de centralité": [alpha_df(centrality_eval), alpha_df(centrality_eval, good_only=True)]}
print("Alphas de Krippendorff:")
pd.DataFrame.from_dict(alphas_krip, orient="index", columns=["toutes les exemples", "sans mauvaises exemples"])

Alphas de Krippendorff:


,toutes les exemples,sans mauvaises exemples
échelle standard,0.117073,0.196759
échelle réduite,0.188813,0.299045
échelle de gravité,0.256008,0.408440
échelle de centralité,0.188519,0.302611


In [177]:
trust_coefs = {'standard_coef':get_trust_coefs(answers),
               'standard_sans_mauvaises_coef':get_trust_coefs(answers, good_only=True),
               'réduit_coef':get_trust_coefs(small),
               'réduit_sans_mauvaises_coef':get_trust_coefs(small, good_only=True),
               'gravité_coef':get_trust_coefs(gravity_eval),
               'gravité_sans_mauvaises_coef':get_trust_coefs(gravity_eval, good_only=True),
               'centralité_coef':get_trust_coefs(centrality_eval),
               'centralité_sans_mauvaises_coef':get_trust_coefs(centrality_eval, good_only=True)}
bad_annot = {scale:list(ident_bad_annot(coef)) for scale, annot in trust_coefs.items()}
if not(any(bad_annot.values())): print("Il n'y a pas d'annotateurs douteaux.")
else:
    print("Annotateurs douteux:")
    display(pd.DataFrame({scale:["❌" if annotator in annot else "" for annotator in answers.index] for scale, annot in bad_annot.items()}, index=answers.index))

Il n'y a pas d'annotateurs douteaux.


In [213]:
info1 = pd.concat([info, pd.DataFrame(trust_coefs)], axis=1, join="inner")
info1.insert(loc=2, column='average', value=info1.loc[:, ~info1.columns.isin(['ling', 'coref'])].mean(axis=1))
display(info1.sort_values(by="average", ascending=False).style.set_properties(**{'background-color': 'floralwhite'}, subset=['average']))

,ling,coref,average,standard_coef,standard_sans_mauvaises_coef,réduit_coef,réduit_sans_mauvaises_coef,gravité_coef,gravité_sans_mauvaises_coef,centralité_coef,centralité_sans_mauvaises_coef
sylvie.billot@univ-orleans.fr,non,oui,0.964644,0.996800,0.957998,0.996895,0.914596,0.949488,0.924558,0.996812,0.980003
loic.grobol@gmail.com,oui,oui,0.920267,0.899060,0.991583,0.851990,0.996768,0.749142,0.936686,0.940185,0.996722
ilaine.wang@inalco.fr,oui,non,0.900124,0.967589,0.937070,0.973415,0.809330,0.997056,0.996635,0.834543,0.685350
ygor.gallina@univ-nantes.fr,non,non,0.898777,0.870167,0.996591,0.835732,0.956970,0.737140,0.980889,0.858850,0.953877
fanny.krimou@hotmail.fr,oui,non,0.880571,0.840049,0.980110,0.795324,0.942989,0.751958,0.973771,0.830783,0.929584
emmanuel.schang@univ-orleans.fr,oui,oui,0.879097,0.839957,0.996591,0.759997,0.956970,0.788120,0.980889,0.756371,0.953877
Frederique.bordignon@enpc.fr,oui,non,0.818699,0.786076,0.937099,0.747536,0.871181,0.708257,0.936222,0.723832,0.839387
bergler@cse.concordia.ca,non,oui,0.808034,0.723526,0.941452,0.692038,0.882177,0.684741,0.980889,0.745925,0.813527
mariya,oui,oui,0.799558,0.877624,0.671122,0.977614,0.632575,0.865560,0.530962,0.987989,0.853020
anais.halftermeyer@univ-orleans.fr,oui,oui,0.710894,0.632276,0.724254,0.674202,0.733764,0.721992,0.807187,0.707675,0.685807


In [11]:
metrics = pd.read_csv("scores.csv", index_col="id").drop("Unnamed: 0", axis=1).drop("name", axis=1)
recalls = metrics.drop("MUC_P", axis=1).drop("MUC_F1", axis=1).drop("B3_P", axis=1).drop("B3_F1", axis=1).drop("CEAFm_P", axis=1).drop("CEAFm_F1", axis=1).drop("CEAFe_P", axis=1).drop("CEAFe_F1", axis=1).drop("BLANC_P", axis=1).drop("BLANC_F1", axis=1).drop("LEA_P", axis=1).drop("LEA_F1", axis=1).drop("CoNLL_2012", axis=1).T
precisions = metrics.drop("MUC_R", axis=1).drop("MUC_F1", axis=1).drop("B3_R", axis=1).drop("B3_F1", axis=1).drop("CEAFm_R", axis=1).drop("CEAFm_F1", axis=1).drop("CEAFe_R", axis=1).drop("CEAFe_F1", axis=1).drop("BLANC_R", axis=1).drop("BLANC_F1", axis=1).drop("LEA_R", axis=1).drop("LEA_F1", axis=1).drop("CoNLL_2012", axis=1).T
fs = metrics.drop("MUC_R", axis=1).drop("MUC_P", axis=1).drop("B3_R", axis=1).drop("B3_P", axis=1).drop("CEAFm_R", axis=1).drop("CEAFm_P", axis=1).drop("CEAFe_R", axis=1).drop("CEAFe_P", axis=1).drop("BLANC_R", axis=1).drop("BLANC_P", axis=1).drop("LEA_R", axis=1).drop("LEA_P", axis=1).T

In [18]:
a_mean = answers[range(1, 128)].astype(float).mean()
a_mode = answers[range(1, 128)].astype(float).mode().mean()
s_mean = small[range(1, 128)].astype(float).mean()*1.75
s_mode = small[range(1, 128)].astype(float).mode().mean()*1.75
gravity_to_num = {"grave":0., "pas grave":1., "pas de fautes":2.}
g_mean = gravity_eval.applymap(lambda x : gravity_to_num[x] if x in gravity_to_num.keys() else x).mean()*3.5
g_mode = gravity_eval.applymap(lambda x : gravity_to_num[x] if x in gravity_to_num.keys() else x).mode().mean()*3.5
bars = {"moyenne":{i:pd.Series([a_mean[i], s_mean[i], g_mean[i], fs[i]["MUC_F1"]*7, fs[i]["B3_F1"]*7, fs[i]["CEAFm_F1"]*7, fs[i]["CEAFe_F1"]*7, fs[i]["BLANC_F1"]*7, fs[i]["LEA_F1"]*7, fs[i]["CoNLL_2012"]*7], index=["standard", "réduit", "gravité_moyenne", "MUC", "B_CUBED", "CEAFm", "CEAFe", "BLANC", "LEA", "CoNLL"]) for i in range(1, 128)}, "mode":{i:pd.Series([a_mode[i], s_mode[i], g_mode[i], fs[i]["MUC_F1"]*7, fs[i]["B3_F1"]*7, fs[i]["CEAFm_F1"]*7, fs[i]["CEAFe_F1"]*7, fs[i]["BLANC_F1"]*7, fs[i]["LEA_F1"]*7, fs[i]["CoNLL_2012"]*7], index=["standard", "réduit", "gravité_moyenne", "MUC", "B_CUBED", "CEAFm", "CEAFe", "BLANC", "LEA", "CoNLL"]) for i in range(1, 128)}}


head = "<table><tbody>"
for bar in bars:
    row = "<tr><th>{}</th>".format(bar)
    for name, series in bars[bar].items():
        s = series.copy()
        s.name=name
        row += "<td>{}</td>".format(s.to_frame().style.bar(align="mid", color='#5fba7d').render())
    row += '</tr>'
    head += row

head+= "</tbody></table>"

print("Comparaison naive (de 0 à 7, des échelles plus petites sont multipliés par des coefficients pour obtenir 7 comme maximum):")
HTML(head)

Comparaison naive (de 0 à 7, des échelles plus petites sont multipliés par des coefficients pour obtenir 7 comme maximum):
